## Import


In [ ]:
%load_ext autoreload
%autoreload 2
from model import train_garch_x, forecast_garch_x, load_features
from feature_engineering import engineer_features
from data_ingestion import get_data
import pandas as pd
import numpy as np
from config import VOL_WINDOW, VOL_TARGET_HORIZON, DATA_RAW, TEST_SIZE,RESULTS_DIR
import matplotlib.pyplot as plt
from data_ingestion import get_sentiment_data


ModuleNotFoundError: No module named 'imp'

In [13]:
df = get_data()
## Add date column to df (needed for merge)
df['date'] = pd.to_datetime(df.index).date
df

Loading existing data from c:\users\hp\financial-volatility-forecasting\data\raw\SPY.csv


,Close,High,Low,Open,Volume,date
Date,,,,,,
1993-01-29,24.313019,24.330311,24.209265,24.330311,1003200,1993-01-29
1993-02-01,24.485966,24.485966,24.330335,24.330335,480500,1993-02-01
1993-02-02,24.537828,24.555121,24.416782,24.468659,201300,1993-02-02
1993-02-03,24.797224,24.814516,24.555131,24.572424,529400,1993-02-03
1993-02-04,24.900984,24.952861,24.607014,24.883691,531500,1993-02-04
...,...,...,...,...,...,...
2025-12-04,684.390015,685.369995,681.340027,685.299988,61970300,2025-12-04
2025-12-05,685.690002,688.390015,684.580017,685.469971,79241000,2025-12-05
2025-12-08,683.630005,686.640015,681.570007,686.590027,55231500,2025-12-08


In [16]:
returns_full = df['Close'].pct_change()  # Use simple returns, convert to log in GARCH
returns_full = np.log(1 + returns_full)  # Log returns

In [17]:
df = engineer_features(
    df,
    sentiment_df,
    VOL_WINDOW,
    VOL_TARGET_HORIZON
)

In [18]:
print(f"Engineered data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Engineered data shape: (8233, 17)
Columns: ['Close', 'High', 'Low', 'Open', 'Volume', 'log_return', 'volatility_20d', 'abs_return', 'return_squared', 'lag_1', 'lag_5', 'lag_10', 'lag_20', 'rolling_abs_return_mean_20d', 'sentiment', 'sentiment_lag1', 'target_volatility']


## Load sentiment feature

In [ ]:
## Load sentiment feature + engineer (put BEFORE split_idx)



df = get_data(live_update=True)  # Live data!
sentiment_df = get_sentiment_data()
df = engineer_features(df, sentiment_df, VOL_WINDOW, VOL_TARGET_HORIZON)

split_idx = int(len(df) * (1 - TEST_SIZE))
    
sentiment = df["sentiment_lag1"]
returns = df["log_return"]
target = df["target_volatility"]


sentiment = df["sentiment_lag1"]
sentiment_train = sentiment.iloc[:split_idx]
sentiment_test = sentiment.iloc[split_idx:]
returns_train = returns.iloc[:split_idx]
returns_test = returns.iloc[split_idx:]
target_test = target.iloc[split_idx:]



In [20]:
print(f"\nTrain size: {len(returns_train)}, Test size: {len(returns_test)}")



Train size: 6586, Test size: 1647


## Rolling GARCH-X loop (parallel to GARCH)

In [ ]:
garch_x_preds = []

for t in range(len(returns) - split_idx):
    r_train = returns.iloc[:split_idx + t]
    x_train = sentiment.iloc[:split_idx + t].values.reshape(-1,1)
    
    model = train_garch_x(r_train, x_train)
    
    x_next = np.array([[sentiment.iloc[split_idx + t]]])

    pred = forecast_garch_x(model, x_next)
    garch_x_preds.append(pred)


c:\users\hp\financial-volatility-forecasting\src\model.py:3: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  from xgboost import XGBRegressor
c:\users\hp\financial-volatility-forecasting\src\model.py:3: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  from xgboost import XGBRegressor
c:\users\hp\financial-volatility-forecasting\src\model.py:3: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  from xgboost import XGBRegressor
c:\users\hp\financial-volatility-forecasting\src\model.py:3: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  from xgboost import XGBRegressor
c:\users

## Evaluate

In [24]:
print(model.params)

Const      -3.487055
x0          0.000000
omega       0.000003
alpha[1]    0.100000
beta[1]     0.880000
Name: params, dtype: float64


In [25]:
target_test = target.iloc[split_idx:]
metrics_x = evaluate_models(y_true=target_test.values, y_pred=garch_x_preds, model_name="GARCH-X")

[autoreload of feature_engineering failed: Traceback (most recent call last):
  File "c:\Users\hp\Financial-volatility-forecasting\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hp\Financial-volatility-forecasting\venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 546, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
  File "C:\Users\hp\anaconda3\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2665: character maps to <undefined>
]


NameError: name 'evaluate_models' is not defined